### 1. Imports

In [1]:
import sys
import os

# En notebooks usamos os.getcwd() en lugar de __file__
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [29]:
import re
from transformers import pipeline
from stdnum.es import nif
from lib.language.regex import nif_detector, name_detector, nif_empresa_detector

### 2. Texto de test

In [27]:
texto_prueba = """ACTA DE REUNIÓN DE PERSONAL - EMPRESA INDUSTRIAS GLOBALES S.A.

En la ciudad de Madrid, a 9 de abril de 2025, siendo las 09:00 horas, se reunieron en la sala de conferencias de la sede central de Industrias Globales S.A. con NIF: A12345671 los siguientes empleados para tratar asuntos relacionados con la reestructuración del departamento de logística:

María del Carmen Rodríguez Pérez, con DNI 49665075W, jefa de operaciones logísticas.

Juan Ignacio Fernández López, con DNI 50439827P, supervisor de almacenes regionales.

Lucía Martínez Hidalgo, con DNI 71234567R, coordinadora de transporte y rutas.

Carlos Alberto Suárez Gómez, con DNI 33112233S, jefe de planificación estratégica.

Ana Sofía Gutiérrez Llamas, con DNI 77654321B, responsable de relaciones sindicales.

Pedro José Álvarez Domínguez, con DNI 12345678C, técnico senior en procesos logísticos.

Elena Beatriz Mendoza Ruiz, con DNI 46579813L, especialista en control de calidad.

José Luis Ramírez Ortega, con DNI 89456123F, representante del comité de empresa.

Laura Isabel Sánchez Mora, con DNI 46090352P, auxiliar administrativa.

Francisco Javier Torres Blanco, con DNI 67389214T, analista de datos operativos.

Durante la sesión, también se mencionaron a otros empleados que no estuvieron presentes pero cuyos casos fueron discutidos:

Mónica Pilar Nieto Salas, DNI 22889977A, actualmente en comisión de servicio en la sede de Valencia.

Miguel Ángel Ruiz Terrón, DNI 55993145D, asignado temporalmente al proyecto internacional de Lisboa.

Sandra Milena Castro Ramírez, DNI 83729164N, quien solicitó excedencia por estudios.

Se estableció que la próxima sesión de revisión operativa se llevará a cabo el 22 de abril de 2025. El acta fue revisada y aprobada por:

Diego Fernando Herrera Solís, DNI 39567812E, director general de logística.

Beatriz Carolina Núñez Rivas, DNI 33273427-V, secretaria ejecutiva del departamento.

Sin más asuntos que tratar, se dio por finalizada la reunión a las 11:45 horas del mismo día."""

### 3. Detect con funciones propias

In [ ]:
print("Nombres detectados (librería propia):")
nombres_propios = name_detector(texto_prueba)
print(nombres_propios)

print("\nNIFs detectados (librería propia):")
nifs_propios = nif_detector(texto_prueba)
print(nifs_propios) # Solo pilla los que se asegure que los num % 23 == letra esperada

print("\nNIFs empresa  detectados (lib propia):")
nif_empresa = nif_empresa_detector(texto_prueba)
print(nif_empresa)

Nombres detectados (librería propia):
['de Madrid', 'de Industrias Globales', 'María del Carmen Rodríguez Pérez', 'Juan Ignacio Fernández López', 'Lucía Martínez Hidalgo', 'Carlos Alberto Suárez Gómez', 'Ana Sofía Gutiérrez Llamas', 'Pedro José Álvarez Domínguez', 'Elena Beatriz Mendoza Ruiz', 'José Luis Ramírez Ortega', 'Laura Isabel Sánchez Mora', 'Francisco Javier Torres Blanco', 'Mónica Pilar Nieto Salas', 'de Valencia', 'Miguel Ángel Ruiz Terrón', 'de Lisboa', 'Sandra Milena Castro Ramírez', 'Diego Fernando Herrera Solís', 'Beatriz Carolina Núñez Rivas']

NIFs detectados (librería propia):
['49665075W', '46090352P', '33273427V']

NIFs detectados (empresa lib propia):
['A12345671']


### 4. Detect de nombres con Bert

In [8]:
# Crea un pipeline para NER usando un modelo en español
ner_pipeline = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")

# Ejecuta la detección de entidades en el texto de prueba
ner_results = ner_pipeline(texto_prueba)

print("Resultados de NER con Hugging Face:")
for entity in ner_results:
    if entity['entity_group'] == 'PER':
        print(entity)

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Resultados de NER con Hugging Face:
{'entity_group': 'PER', 'score': np.float32(0.9997549), 'word': 'María del Carmen Rodríguez Pérez', 'start': 335, 'end': 367}
{'entity_group': 'PER', 'score': np.float32(0.9996547), 'word': 'Juan Ignacio Fernández López', 'start': 421, 'end': 449}
{'entity_group': 'PER', 'score': np.float32(0.9997978), 'word': 'Lucía Martínez Hidalgo', 'start': 507, 'end': 529}
{'entity_group': 'PER', 'score': np.float32(0.99979824), 'word': 'Carlos Alberto Suárez Gómez', 'start': 587, 'end': 614}
{'entity_group': 'PER', 'score': np.float32(0.99975026), 'word': 'Ana Sofía Gutiérrez Llamas', 'start': 671, 'end': 697}
{'entity_group': 'PER', 'score': np.float32(0.99962443), 'word': 'Pedro José Álvarez Domínguez', 'start': 757, 'end': 785}
{'entity_group': 'PER', 'score': np.float32(0.99977684), 'word': 'Elena Beatriz Mendoza Ruiz', 'start': 846, 'end': 872}
{'entity_group': 'PER', 'score': np.float32(0.9996145), 'word': 'José Luis Ramírez Ortega', 'start': 930, 'end': 

### 5. Detect de NIF con python-stdnum

In [25]:
print("\nNIFs validados con python-stdnum:")
# Extraer posibles NIFs con regex básica (para pasar al validador)
posibles_nifs = re.findall(r'\d{8}[A-Z]', texto_prueba)

nifs_validos = []
for posible_nif in posibles_nifs:
    if nif.is_valid(posible_nif):
        nifs_validos.append(posible_nif)

print(nifs_validos)


NIFs validados con python-stdnum:
['49665075W']


### 6. Comparacion

In [33]:
print("\n--- COMPARACIÓN ---")
print(f"Nombres detectados (propios): {len(nombres_propios)}")
print(f"Nombres detectados (NER): {len([entity for entity in ner_results if entity['entity_group'] == 'PER'])}")
print(f"NIFs detectados (propios): {len(nifs_propios)}")
print(f"NIFs válidos (stdnum): {len(nifs_validos)}")
print(f"NIFs empresa detectados (propios): {len(nif_empresa)}")
print(f"Nombres de empresa detectados (NER): {len([entity for entity in ner_results if entity['entity_group'] == 'ORG'])}")
print(f"Nombre de empresa segun NER: {[entity['word'] for entity in ner_results if entity['entity_group'] == 'ORG']}")


--- COMPARACIÓN ---
Nombres detectados (propios): 19
Nombres detectados (NER): 16
NIFs detectados (propios): 3
NIFs válidos (stdnum): 1
NIFs empresa detectados (propios): 1
Nombres de empresa detectados (NER): 1
Nombre de empresa segun NER: ['Industrias Globales S. A']


## Resolucion

A continuación, se explica la estrategia elegida para extraer información del texto:

1. Detección de nombres de personas:

Se utilizará un modelo NER (por ejemplo, con Hugging Face) para identificar entidades del tipo "PER". Este método estadístico es robusto al contexto y las variaciones en la forma en que se escriben los nombres.
Posteriormente, se ejecutará la función propia name_detector (basada en expresiones regulares o reglas) para confirmar o complementar los nombres identificados. Esta combinación permite mejorar la precisión de la extracción al aprovechar lo mejor de ambos enfoques.

2. Extracción de DNI:

Para obtener los DNI, se opta por emplear únicamente la función nif_detector. Esta función ha demostrado ser más efectiva que una solución basada en expresiones regulares simples, ya que no solo extrae el patrón, sino que también valida el dígito de control según la norma.

2. Detección del nombre de empresa:

La identificación del nombre de la empresa se realizará utilizando el mismo modelo NER, pero filtrando aquellas entidades clasificadas como "ORG". Este método permite detectar de manera más confiable nombres de organizaciones, dada la complejidad y variedad de formatos que pueden presentar.

Esta solución híbrida aprovecha la capacidad de los modelos de lenguaje para contextualizar y detectar entidades relevantes (nombres de persona y empresas) mientras se confía en una función especializada y verificada para la extracción y validación de DNI.